In [1]:
%matplotlib widget
import geopandas as gpd
import logging
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import pyvista as pv

from georefcam import ASCIIGridDEM, CTCameraModel, GeoRefCam
from pathlib import Path
from PIL import Image

# logging.basicConfig(level=logging.DEBUG)

# Définition des dossiers contenant les données nécessaires à l'exécution du notebook (À ÉDITER)

In [2]:
carto_dir = Path("/home/florent/git_repos/d4g/pyronear/explo/cartos/BDALTIV2_2-0_25M_ASC_LAMB93-IGN69_D007_2022-12-16/BDALTIV2")
camdata_dir = Path("/home/florent/git_repos/d4g/pyronear/explo/pyronear_cam_data")
donnees_dir = Path("1_DONNEES_LIVRAISON_2023-01-00224/BDALTIV2_MNT_25M_ASC_LAMB93_IGN69_D007")
dalles_dir = Path("3_SUPPLEMENTS_LIVRAISON_2023-01-00224/BDALTIV2_MNT_25M_ASC_LAMB93_IGN69_D007/")

cam_name = "brison_4"
test_img_name = "pyronear_brison_4_2023_07_04T06_07_57.jpg"
gcp_filepath = "df_annotations.pkl"

# Création d'un modèle de caméra orientée et localisée avec `cameratransform`

Création d'un geodataframe des dalles dans la carto contenant une caméra (problème si les dalles recherchées sont à cheval sur deux cartos, cas relativement improbable d'une caméra couvrant la bordure entre deux départements) :

In [3]:
dalles_shape_file = Path("dalles.shp")
gdf_dalles = gpd.read_file(carto_dir / dalles_dir / dalles_shape_file)

Création d'un geodataframe des caméras utilisées à l'aide du fichier de référence 'API_DATA - devices.csv' :

In [4]:
cam_file = camdata_dir / "API_DATA - devices.csv"
df_cams = pd.read_csv(cam_file)
gdf_cams = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_cams.lon, df_cams.lat, crs="WGS84"), data=df_cams)

Sélection d'une caméra et récupération de ses informations, sélection du fichier de dalle contenant sa localisation

In [5]:
cam_info = gdf_cams[gdf_cams.login == cam_name].squeeze()
test_img_path = camdata_dir / cam_name / test_img_name
test_img = Image.open(test_img_path)
test_img_res = test_img.size

gdf_dalles_wgs84 = gdf_dalles.to_crs("WGS84")
dalle_cam = gdf_dalles[gdf_dalles_wgs84.geometry.contains(cam_info.geometry)].squeeze()
nom_dalle = dalle_cam.NOM_DALLE
dalle_file = carto_dir / donnees_dir / Path(f"{nom_dalle}.asc")

# Évaluation de la projection d'un ensemble de GCP

In [6]:
with open(gcp_filepath, "rb") as f:
    df_annot_gcp = pickle.load(f)
df_annot_gcp = df_annot_gcp.rename({"ele": "alt"}, axis=1).astype({"posx": int, "posy": int, "lat": float, "lon": float, "alt": float})
df_annot_gcp
gdf_annot_gcp = gpd.GeoDataFrame(df_annot_gcp, geometry=gpd.points_from_xy(df_annot_gcp.lon, df_annot_gcp.lat, df_annot_gcp.alt), crs="WGS84")
gdf_annot_gcp

,posx,posy,lat,lon,alt,geometry
0,651,328,44.609620,4.098371,1448.0,POINT Z (4.09837 44.60962 1448.00000)
1,789,319,44.612899,4.126470,1458.0,POINT Z (4.12647 44.61290 1458.00000)
2,978,409,44.577702,4.188103,643.0,POINT Z (4.18810 44.57770 643.00000)
3,1190,310,44.612280,4.185187,1316.0,POINT Z (4.18519 44.61228 1316.00000)


Sélection d'un GCP et génération de son rayon de projection dans le repère monde

In [7]:
dem = ASCIIGridDEM(dalle_file, gdf_dalles.crs)
dem.build_pcd(sample_step=1)
dem.build_mesh()

In [8]:
device_params = {"image": test_img_res, "view_x_deg": 87, "view_y_deg": 44}
ypr_orientation_params = {"elevation_m": cam_info.elevation, "pitch_deg": cam_info.pitch, "roll_deg": 0, "yaw_deg": cam_info.azimuth}
location_params = {"lat": cam_info.lat, "lon": cam_info.lon, "alt": cam_info.elevation}

cam_model = CTCameraModel(device_params, ypr_orientation_params, location_params)

In [9]:
geocam = GeoRefCam(cam_model, dem)

In [10]:
test_gcp_idx = [2, 2]
pixel_points = gdf_annot_gcp.loc[test_gcp_idx, ["posx", "posy"]]
# pixel_points = gdf_annot_gcp[["posx", "posy"]]

rays = geocam.camera_model.project_pixel_points_to_world_rays(pixel_points)
rays

array([[[ 44.54515457,  44.67704565],
        [  4.21653414,   4.14014126],
        [780.        ,   0.        ]],

       [[ 44.54515457,  44.67704565],
        [  4.21653414,   4.14014126],
        [780.        ,   0.        ]]])

In [11]:
filtered_inter_points, filtered_inter_triangles, df_ray = geocam.cast_rays(rays, check_crs=True, min_d_intersection_m=0, return_df_ray=True)

In [12]:
df_ray

,ori_x,ori_y,ori_z,dest_x,dest_y,dest_z,inter_x,inter_y,inter_z,n_tri,dist_o
n_ray,,,,,,,,,,,
0,796636.658666,6.383642e+06,780.0,790357.216371,6.398201e+06,0.0,797117.499813,6.387535e+06,780.0,1770966,3922.152891
1,796636.658666,6.383642e+06,780.0,790357.216371,6.398201e+06,0.0,797117.499813,6.387535e+06,780.0,1770966,3922.152891


In [13]:
import numpy as np

plot_margin_m = 1000

target_points = np.array([list(gdf_annot_gcp.to_crs(gdf_dalles.crs).loc[idx, "geometry"].coords)[0] for idx in test_gcp_idx])
origins = df_ray[["ori_x", "ori_y", "ori_z"]].values
inter_points = df_ray[["inter_x", "inter_y", "inter_z"]].values

print(f"target_points:\n{target_points}\n\norigins:\n{origins}\n\ninter_points:\n{inter_points}")

points_of_interest = np.vstack((target_points, origins, inter_points))
(x_min, y_min, _) = points_of_interest.min(axis=0) - plot_margin_m
(x_max, y_max, _) = points_of_interest.max(axis=0) + plot_margin_m

alts_meshgrid = dem.pcd
zone_mask_idx = np.vstack(((alts_meshgrid[:, :, 0] > x_min) & (alts_meshgrid[:, :, 0] < x_max) & (alts_meshgrid[:, :, 1] > y_min) & (alts_meshgrid[:, :, 1] < y_max)).nonzero())
x_min_idx, y_min_idx = zone_mask_idx.min(axis=1)
x_max_idx, y_max_idx = zone_mask_idx.max(axis=1)
plot_meshgrid = alts_meshgrid[x_min_idx:x_max_idx, y_min_idx:y_max_idx, :]

plot_pv_meshgrid = pv.StructuredGrid(*[plot_meshgrid[:, :, i] for i in range(3)])
plot_pv_meshgrid["alt"] = plot_meshgrid[:, :, 2].ravel(order="F")  # add the altitude as a scalar field in order to plot it as a colormap


plotter = pv.Plotter(lighting="none")
plotter.add_mesh(plot_pv_meshgrid, smooth_shading=True, specular=0.5, specular_power=15)
light = pv.Light()
light.set_direction_angle(45, -20)
plotter.add_light(light)

bar_height_m = round(max(abs(alts_meshgrid[0, x_max_idx, 0] - alts_meshgrid[0, x_min_idx, 0]), abs(alts_meshgrid[y_max_idx, 0, 1] - alts_meshgrid[y_min_idx, 0, 1])) / 10)

targets_pl = [pv.Line(target_point, target_point + np.array([0, 0, bar_height_m])) for target_point in target_points]
origins_pl = [pv.Line(origin, origin + np.array([0, 0, bar_height_m])) for origin in origins]
inters_pl = [pv.Line(inter_point, inter_point + np.array([0, 0, bar_height_m])) for inter_point in inter_points]
ray_cuts = [pv.Line(origin, inter_point) for origin, inter_point in zip(origins, inter_points)]
sep_lines = [pv.Line([target_point[0], target_point[1], max(target_point[2], inter_point[2])], [inter_point[0], inter_point[1], max(target_point[2], inter_point[2])]) for target_point, inter_point in zip(target_points, inter_points)]
intersections = [pv.PolyData(inter_point) for inter_point in inter_points]

for n_proj in range(len(target_points)):
    plotter.add_mesh(origins_pl[n_proj], color="red", line_width=2, opacity=1)
    plotter.add_mesh(ray_cuts[n_proj], color="orange", line_width=2, opacity=1)
    plotter.add_mesh(inters_pl[n_proj], color="lightgreen", line_width=2, opacity=1)
    plotter.add_mesh(targets_pl[n_proj], color="yellow", line_width=2, opacity=1)
    plotter.add_mesh(sep_lines[n_proj], color="cyan", line_width=2, opacity=1,)
    plotter.add_mesh(intersections[n_proj], color="maroon", point_size=5)

plotter.show()

target_points:
[[7.943240e+05 6.387223e+06 6.430000e+02]
 [7.943240e+05 6.387223e+06 6.430000e+02]]

origins:
[[7.96636659e+05 6.38364234e+06 7.80000000e+02]
 [7.96636659e+05 6.38364234e+06 7.80000000e+02]]

inter_points:
[[7.97117500e+05 6.38753491e+06 7.80000000e+02]
 [7.97117500e+05 6.38753491e+06 7.80000000e+02]]


Widget(value='<iframe src="http://localhost:38897/index.html?ui=P_0x7f0c383317e0_0&reconnect=auto" class="pyvi…

In [14]:
# """
# Ray Tracing
# ~~~~~~~~~~~

# Single line segment ray tracing for PolyData objects.
# """

# import pyvista as pv
# # Create source to ray trace
# sphere = pv.Sphere(radius=0.85)

# # Define line segment
# start = [0, 0, 0]
# stop = [0.25, 1, 0.5]

# # Perform ray trace
# points, ind = sphere.ray_trace(start, stop)

# # Create geometry to represent ray trace
# ray = pv.Line(start, stop)
# intersection = pv.PolyData(points)

# # Render the result
# p = pv.Plotter(off_screen=True)
# p.add_mesh(sphere,
#            show_edges=True, opacity=0.5, color="w",
#            lighting=False, label="Test Mesh")
# p.add_mesh(ray, color="blue", line_width=5, label="Ray Segment")
# p.add_mesh(intersection, color="maroon",
#            point_size=25, label="Intersection Points")
# p.add_legend()
# p.show()


# ###############################################################################
# # Vectorised Ray Tracing
# # +++++++++++
# #
# # Perform many ray traces simultaneously with a PolyData Object
# # (requires optional dependencies trimesh, rtree and pyembree)
# #

# from math import sin, cos, radians
# # Create source to ray trace
# sphere = pv.Sphere(radius=0.85)

# # Define a list of origin points and a list of direction vectors for each ray
# vectors = [ [cos(radians(x)), sin(radians(x)), 0.01 * x] for x in range(0, 360, 5)]
# origins = [[0, 0, 0]] * len(vectors)

# # Perform ray trace
# points, ind_ray, ind_tri = sphere.multi_ray_trace(origins, vectors)

# # Create geometry to represent ray trace
# rays = [pv.Line(o, v) for o, v in zip(origins, vectors)]
# intersections = pv.PolyData(points)

# # Render the result
# p = pv.Plotter()
# p.add_mesh(sphere,
#            show_edges=True, opacity=0.5, color="w",
#            lighting=False, label="Test Mesh")
# p.add_mesh(rays[0], color="blue", line_width=5, label="Ray Segments")
# for ray in rays[1:]:
#     p.add_mesh(ray, color="blue", line_width=5)
# p.add_mesh(intersections, color="maroon",
#            point_size=25, label="Intersection Points")
# p.add_legend()
# p.show()
